# Sample of dataframes required for pyfolio integration

> DISCLAIMER: Yes, zipline is a thing, but I don't think it's conducive to the type of backtesting I'm doing. I can also only get it working with Docker as mac osx deploy is broken right now due to some tricky dependency issues. I might come back to zipline, but for now, going to try and replicate the results that zipline produces and tie it into pyfolio

In [ ]:
import arrow
import pandas as pd
import numpy as np

import gzip
import math
import sys

sys.path.append('../src')

import quality_momentum as qm

In [ ]:
transactions = pd.read_csv(gzip.open('./pyfolio_data/test_txn.csv.gz'), index_col=0, parse_dates=True)
positions = pd.read_csv(gzip.open('./pyfolio_data/test_pos.csv.gz'), index_col=0, parse_dates=True)
returns = pd.read_csv(gzip.open('./pyfolio_data/test_returns.csv.gz'), index_col=0, parse_dates=True, header=None)[1]

## Transactions dataframe

A record is inserted for every transaction completed.

In [ ]:
transactions.info()

In [ ]:
def foo():
    return 23.34
# sum up the amount grouped by symbol in the transactions dataframe
stock_positions = transactions.groupby('symbol').amount.sum()
# filter out the rows with amount = 0
df = stock_positions[stock_positions >= 0].to_frame()
df["amount"] = df["amount"] * -1
df["price"] = df.apply(lambda x: foo(), axis=1)
df["txn_dollars"] = df["price"] * df["amount"] * -1
df["date"] = pd.to_datetime(arrow.get(trading_day.date()).datetime)
df.set_index("date", inplace=True)


In [ ]:
df

In [ ]:
transactions.head(20)
# transactions[transactions['symbol'] == 'AMD']

## Positions dataframe

Any position we have is a column in this datetime-based index. The total amount of each equity is tracked through time.

In [ ]:
positions.info()

In [ ]:
trading_day = arrow.get('2009-12-30')
latest_positions = positions.index[-1]
latest_row = positions.loc[[latest_positions]]
# stow away the available cash we have and remove it from positions
starting_cash = latest_row['cash'][0]
del latest_row['cash']
# filter out columns with a value of 0


# transactions should look something like this
# amount	price	    symbol	txn_dollars
# -697	    4.937500	CERN	3441.437500

In [ ]:
latest_row

In [ ]:
returns.index

In [ ]:
positions.loc[['2004-01-09']].sum(axis=1)[0]

In [ ]:
positions.loc[['2004-01-09']]

In [ ]:
daily_total_position = positions.sum(axis=1)

In [ ]:
calculated_returns = daily_total_position.pct_change().fillna(0)

In [ ]:
print(calculated_returns.index, calculated_returns.dtype)

## Returns series

A series that calculates daily returns over the same time period

In [ ]:
print(returns.index, returns.dtype)

In [ ]:
returns

In [ ]:
returns[55:65]

In [ ]:
calculated_returns[50:60]

## Time to generate some transactions!

Concepts to figure out:
- portfolio management
- capital allocation
- market-cap weighted vs equal weighted
- how to roll through a period of time

In [ ]:
portfolio = qm.portfolio.Portfolio(
    start_date=arrow.get('2018-01-01'), 
    end_date=arrow.get('2018-12-01'), 
    capital_allocation=100000.0,
    weighting=qm.portfolio.WeightType.equal_weighted)

In [ ]:
portfolio.run()

In [ ]:
portfolio.transactions.head(20)